In [ ]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import pandas as pd

In [ ]:
# Load Train dataset
data = np.loadtxt('train.in')
# helpfulness |3| 4| 5| 6| 7| 8| 9| 10| 11| 12-31| 32| 33| 34
Data_X = data[:, 3:33]
Data_Y = data[:, 36]

train_X, test_X, train_y, test_y = train_test_split(Data_X, Data_Y, test_size=0.2)

In [ ]:
first_model = RandomForestRegressor(n_estimators = 300)
first_model.fit(train_X, train_y)
y_first_model = first_model.predict(test_X)
mse = mean_squared_error(test_y, y_first_model)
print("MSE : {}".format(mse))

In [ ]:
# Load Test dataset
data = np.loadtxt('test.in')
# helpfulness | 7| 8| 9| 10| 11| 12-31| 32| 33| 34
test = data[:, 3:33]
y_predict = first_model.predict(test)

# clarity | 3| 4| 5| 6| 9| 10| 11| 12-31|32 |33| 35
clar_test1 = data[:, 3:7]
clar_test2 = data[:, 9:33]
clar_test = np.concatenate((clar_test1, clar_test2), axis=1)

In [ ]:
# Read in the data with pandas http://pandas.pydata.org
data = pd.read_csv('train.csv')
data['comments'] = data['comments'].fillna('')
train, test = train_test_split(data)
# Make a pipeline to do unigrams then run linear regression
# http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
pipeline = Pipeline([
        ('cv', HashingVectorizer(ngram_range=(1,1))),
        ('regression', RandomForestRegressor(max_depth=50,n_jobs=-1))
])
cv = GridSearchCV(
    pipeline, {}
).fit(train['comments'], train['quality'])
mse = mean_squared_error(test['quality'], cv.predict(test['comments']))
print("MSE: {}".format(mse))
kaggle_test = pd.read_csv('test.csv')
kaggle_test['comments'] = kaggle_test['comments'].fillna('')
predictions = cv.predict(kaggle_test['comments'])

In [ ]:
y_quality = [0 for i in range(len(predictions))]
outfile = open('predict.out','wb')
for i in range(len(predictions)):
    y_quality[i] = y_predict[i] * 0.2 + 0.8 * predictions[i]
    outfile.write(str(y_quality[i]) + '\n')
outfile.close()

In [ ]:
submit = pd.read_csv('submission1.csv')
submit['quality'] = y_quality
submit.to_csv('submission.csv',index=False)

In [ ]:
# Save the model for later
import pickle
with open('modelCV.pkl', 'wb') as f:
    pickle.dump(cv, f)